<a href="https://colab.research.google.com/github/cianc/AoC2023/blob/main/day03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
TEST = False #@param {type: "boolean"}
TEST_INPUT = 'test_1.txt'
INPUT = 'input_1.txt'



In [ ]:
import string
import re
from google.colab import drive
from typing import NamedTuple

SYMBOLS = set([c for c in string.printable if c not in string.ascii_letters + string.digits + '.'])

class Part(NamedTuple):
  row: int
  start_col: int
  end_col: int
  value: int

class Gear(NamedTuple):
  row: int
  col: int
  parts: list[Part] = []
  ratio: int = 0

def read_schematic() -> list[str]:
  input = TEST_INPUT if TEST else INPUT
  with open(input, 'r') as f:
    rows = f.read().splitlines()
  return rows

def get_parts_and_gears(schematic: list[str]) -> tuple[list[Part], list[Gear]]:
  # First get the parts, then the gears. Do this in two passes for simplicity.
  parts = []
  max_row = len(schematic) - 1
  max_col = len(schematic[0]) - 1
  for row_index, row in enumerate(schematic):
    matches = re.finditer(r'\d+', row)
    for match in matches:
      part = Part(row=row_index, start_col=match.start(0), end_col=match.end(0)-1, value=int(match.group(0)))

      above_row_index = row_index-1 if row_index > 0 else None
      below_row_index = row_index+1 if row_index < max_row else None
      leftmost_col_index = max(part.start_col - 1, 0)
      rightmost_col_index = min(part.end_col + 1, max_col)

      if (
          (above_row_index and (set(schematic[above_row_index][leftmost_col_index:rightmost_col_index+1]).intersection(SYMBOLS))) or
          (below_row_index and (set(schematic[below_row_index][leftmost_col_index:rightmost_col_index+1]).intersection(SYMBOLS))) or
          row[leftmost_col_index] in SYMBOLS or
          row[rightmost_col_index] in SYMBOLS):
        parts.append(part)

  gears = []
  for row_index, row in enumerate(schematic):
    matches = re.finditer(r'\*', row)
    for match in matches:
      col_index = match.start(0)
      gear_parts = []
      for part in parts:
        if part.row in range(row_index - 1, row_index + 2) and col_index in range(part.start_col - 1, part.end_col + 2):
          gear_parts.append(part)
      if len(gear_parts) == 2:
        gear = Gear(row=row_index, col=col_index, parts=gear_parts, ratio = gear_parts[0].value * gear_parts[1].value)
        gears.append(gear)

  return (parts, gears)


schematic = read_schematic()
parts, gears = get_parts_and_gears(schematic)
print(f'parts sum: {sum(p.value for p in parts)}')
print(f'gears sum: {sum(g.ratio for g in gears)}')

